In [32]:
import tensorflow as tf
from tensorflow import keras

import os
from os import path

import tempfile
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
pd.options.mode.chained_assignment = None

import seaborn as sns

import sklearn

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Set DataFrame name:

In [34]:
pca_prec = 95

df_name = f"{pca_prec}"
print(f"Name: {df_name}")

df_path = f"/content/drive/MyDrive/uni/RoboDoc/ml/{df_name}"

if path.exists(df_path) == False:
  os.mkdir(df_path)

Name: 95


In [35]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [36]:
        file = tf.keras.utils
raw_df = pd.read_csv("/content/drive/MyDrive/uni/RoboDoc/db/admissions.csv")
raw_df.head()

,patient_id,hadm_id,icustay_id,hospstay_seq,icustay_seq,age,gender,weight,height,meanbp_mean,meanbp_min,meanbp_max,resprate_min,resprate_max,resprate_mean,tempc_mean,glucose_min,glucose_max,glucose_mean,symptoms,patient_history,diagnoses,length_of_stay_hospital,number_of_icu_stays,length_of_stay_icu,total_length_of_stay_icu,days_to_death,died_in_hospital
0,9973,100020,282580,5,1,58,M,96.0,175.0,87.0,56.0,151.0,8.0,27.0,15.0,35.416667,79.0,101.0,90.0,NaN,V1302,5990;34409;2761;5601;5849;0417;4280;59654;340;...,10.61,1,1.10,1.101088,5.0,f
1,29971,100021,252772,1,1,54,M,NaN,NaN,94.0,69.0,113.0,12.0,27.0,21.0,36.953704,142.0,155.0,146.0,78009;78820;79902,V1581,4019;04104;0413;5728;2819;8248;99649;5712;3030...,60.01,2,2.12,3.621227,28.0,f
2,77221,100024,201668,1,1,71,M,91.0,175.0,76.0,65.0,96.0,11.0,27.0,18.0,36.521164,103.0,176.0,140.0,NaN,V4960,41401;4111;99662;5119;5180;4019;42731;45182;27401,6.33,1,1.08,1.079884,NaN,f
3,29633,100050,279769,1,1,69,M,87.0,183.0,80.0,56.0,109.0,6.0,28.0,18.0,37.362069,106.0,155.0,130.0,NaN,V4501,4280;99811;5990;4241;41401;2875;99791;42731;53081,6.02,1,1.09,1.086192,NaN,f
4,17261,100052,287540,1,1,44,F,NaN,NaN,71.0,50.0,84.0,16.0,24.0,19.0,36.305555,131.0,183.0,150.0,7018;72252,NaN,6111;2768;5533,2.01,1,1.45,1.452083,NaN,f


## Clean null values:

In [37]:
raw_df.describe()

,patient_id,hadm_id,icustay_id,hospstay_seq,icustay_seq,age,weight,height,meanbp_mean,meanbp_min,meanbp_max,resprate_min,resprate_max,resprate_mean,tempc_mean,glucose_min,glucose_max,glucose_mean,length_of_stay_hospital,number_of_icu_stays,length_of_stay_icu,total_length_of_stay_icu,days_to_death
count,49276.000000,49276.000000,49276.000000,49276.000000,49276.0,49276.000000,42680.000000,19358.000000,48728.000000,48728.000000,48728.000000,48672.000000,48672.000000,48672.000000,47899.000000,47996.000000,47996.000000,47996.000000,49276.000000,49276.000000,49276.000000,49276.000000,21626.000000
mean,36830.670387,149981.859830,250083.991152,1.475627,1.0,74.371662,80.713730,169.903916,78.045969,58.483890,104.793486,12.321048,27.145299,18.770607,36.820450,107.717615,201.073631,142.067089,10.001152,1.073362,4.066593,4.463190,12.624665
std,29043.766195,28915.694377,28907.221643,1.617056,0.0,55.318405,24.607055,14.036608,11.346109,13.699139,26.013993,3.729969,6.515951,4.002100,0.629393,35.165880,4564.797559,653.460930,10.824429,0.304267,5.985749,6.729425,9.040974
min,3.000000,100001.000000,200001.000000,1.000000,1.0,14.000000,1.000000,107.000000,17.000000,0.000000,23.000000,0.200000,8.000000,7.000000,30.583333,0.830000,21.000000,21.000000,0.000000,1.000000,0.000000,0.000289,-3.000000
25%,13300.500000,124911.750000,225039.750000,1.000000,1.0,52.000000,65.000000,163.000000,70.000000,51.000000,90.000000,10.000000,23.000000,16.000000,36.438889,86.000000,126.000000,111.000000,4.040000,1.000000,1.200000,1.237046,5.000000
50%,26709.500000,150055.500000,250130.500000,1.000000,1.0,65.000000,78.000000,170.000000,77.000000,59.000000,100.000000,12.000000,26.000000,18.000000,36.793651,103.000000,160.000000,129.000000,6.920000,1.000000,2.120000,2.214132,12.000000
75%,60230.250000,175055.500000,275154.750000,1.000000,1.0,77.000000,92.000000,178.000000,85.000000,66.000000,113.000000,14.000000,30.000000,21.000000,37.187500,124.000000,205.000000,156.000000,11.950000,1.000000,4.110000,4.569068,20.000000
max,99999.000000,199999.000000,299999.000000,41.000000,1.0,311.000000,1251.000000,485.000000,154.000000,125.000000,299.000000,43.000000,69.000000,46.000000,40.833333,592.000000,999999.000000,142967.000000,294.660000,7.000000,153.930000,260.708877,30.000000


In [38]:
df = raw_df.copy()
df.pop('patient_id')
df.pop('hadm_id')
df.pop('icustay_id')
df.pop('hospstay_seq')
df.pop('icustay_seq')

df.dropna(thresh=12, inplace=True) 

df.age[raw_df.age > 90] = 90
df['age'].fillna((df['age'].mean()), inplace=True)

df.gender[raw_df.gender == 'M'] = 1
df.gender[raw_df.gender == 'F'] = 0
df['gender'].fillna((df['gender'].mean()), inplace=True)

df.weight[raw_df.weight > 300] = 300
df['weight'].fillna((df['weight'].mean()), inplace=True)

df.pop('height')
# df['height'].fillna((df['height'].mean()), inplace=True)

df['meanbp_mean'].fillna((df['meanbp_mean'].mean()), inplace=True)
df['meanbp_min'].fillna((df['meanbp_min'].mean()), inplace=True)
df['meanbp_max'].fillna((df['meanbp_max'].mean()), inplace=True)

df['resprate_min'].fillna((df['resprate_min'].mean()), inplace=True)
df['resprate_max'].fillna((df['resprate_max'].mean()), inplace=True)
df['resprate_mean'].fillna((df['resprate_mean'].mean()), inplace=True)

df['tempc_mean'].fillna((df['tempc_mean'].mean()), inplace=True)

df['glucose_min'].fillna((df['glucose_min'].mean()), inplace=True)
df.glucose_max[raw_df.glucose_max > 1000] = 1000
df['glucose_max'].fillna((df['glucose_max'].mean()), inplace=True)
df.glucose_mean[raw_df.glucose_mean > 1000] = 1000
df['glucose_mean'].fillna((df['glucose_mean'].mean()), inplace=True)

# df = df[df.patient_history == df.patient_history]

symptoms_raw = df.symptoms
df.pop('symptoms')
diagnoses_raw = df.diagnoses
df.pop('diagnoses')
patient_history_raw = df.patient_history
df.pop('patient_history')

#df['length_of_stay_hospital'].fillna((df['length_of_stay_hospital'].mean()), inplace=True)
#df['length_of_stay_icu'].fillna((df['length_of_stay_icu'].mean()), inplace=True)                                     

df.pop('number_of_icu_stays')
df.pop('length_of_stay_icu')
#df.pop('days_to_death')
#df[df.days_to_death != df.days_to_death].days_to_death = 0

df = df[df.length_of_stay_hospital == df.length_of_stay_hospital]

df.pop('total_length_of_stay_icu')

df.died_in_hospital[raw_df.died_in_hospital == 't'] = 1
df.died_in_hospital[raw_df.died_in_hospital == 'f'] = 0
df = df[df.died_in_hospital == df.died_in_hospital]

print(df.columns)

Index(['age', 'gender', 'weight', 'meanbp_mean', 'meanbp_min', 'meanbp_max',
       'resprate_min', 'resprate_max', 'resprate_mean', 'tempc_mean',
       'glucose_min', 'glucose_max', 'glucose_mean', 'length_of_stay_hospital',
       'days_to_death', 'died_in_hospital'],
      dtype='object')


In [39]:
df.head()[df.days_to_death == df.days_to_death][df.days_to_death == df.days_to_death]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,age,gender,weight,meanbp_mean,meanbp_min,meanbp_max,resprate_min,resprate_max,resprate_mean,tempc_mean,glucose_min,glucose_max,glucose_mean,length_of_stay_hospital,days_to_death,died_in_hospital
0,58,1,96.000000,87.0,56.0,151.0,8.0,27.0,15.0,35.416667,79.0,101.0,90.0,10.61,5.0,0
1,54,1,80.650038,94.0,69.0,113.0,12.0,27.0,21.0,36.953704,142.0,155.0,146.0,60.01,28.0,0


In [40]:
df.describe()

,age,gender,weight,meanbp_mean,meanbp_min,meanbp_max,resprate_min,resprate_max,resprate_mean,tempc_mean,glucose_min,glucose_max,glucose_mean,length_of_stay_hospital,days_to_death
count,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,21444.000000
mean,63.444870,0.562129,80.650038,78.045969,58.483890,104.793486,12.321048,27.145299,18.770607,36.820450,107.712850,179.918742,139.110810,9.990334,12.628474
std,17.342315,0.496130,21.728365,11.330077,13.679782,25.977234,3.722557,6.503003,3.994148,0.623134,34.844774,85.745079,44.263461,10.812359,9.036426
min,14.000000,0.000000,1.000000,17.000000,0.000000,23.000000,0.200000,8.000000,7.000000,30.583333,0.830000,21.000000,21.000000,0.000000,-3.000000
25%,52.000000,0.000000,67.000000,70.000000,51.000000,90.000000,10.000000,23.000000,16.000000,36.444444,87.000000,126.000000,111.000000,4.040000,5.000000
50%,65.000000,1.000000,80.650038,77.000000,59.000000,100.000000,12.000000,26.000000,18.000000,36.805556,104.000000,161.000000,130.000000,6.910000,12.000000
75%,77.000000,1.000000,90.000000,85.000000,66.000000,113.000000,14.000000,30.000000,21.000000,37.177778,123.000000,204.000000,155.000000,11.930000,20.000000
max,90.000000,1.000000,300.000000,154.000000,125.000000,299.000000,43.000000,69.000000,46.000000,40.833333,592.000000,1000.000000,1000.000000,294.660000,30.000000


## Examine the class label imbalance:

In [41]:
neg, pos = np.bincount(df['died_in_hospital'])
total = neg + pos
print(f'Examples:\n    Total: {total}\n    Positive: {pos} ({100 * pos / total:.2f}% of total)\n')

Examples:
    Total: 48866
    Positive: 5612 (11.48% of total)



## Convert diagnoses:

In [43]:
def load_diagnose_data():
    symptoms = []
    history = []
    diagnoses = []

    with open('/content/drive/MyDrive/uni/RoboDoc/db/symptoms_dict.csv', 'r') as f:
      for line in f:
        split = line.strip("\n").split(",")
        symptoms.append([split[0], split[1], split[2]])
            
    with open('/content/drive/MyDrive/uni/RoboDoc/db/patient_history_dict.csv', 'r') as f:
      for line in f:
        split = line.strip("\n").split(",")
        history.append([split[0], split[1], split[2]])

    with open('/content/drive/MyDrive/uni/RoboDoc/db/diagnoses_dict.csv', 'r') as f:
      for line in f:
        split = line.strip("\n").split(",")
        diagnoses.append([split[0], split[1], split[2]])
            
    return np.asarray(symptoms[1:]), np.asarray(history[1:]), np.asarray(diagnoses[1:])


def diagnoses_to_matrix(symptoms, patient_history, diagnoses, symptoms_dict, patient_history_dict, diagnoses_dict):
    Xs = np.zeros((len(diagnoses), len(symptoms_dict)))
    
    for i, ss in enumerate(symptoms):
        lst = str(ss).split(';')

        for s in lst:
            Xs[i][[symptoms_dict == s]] = 1

            
    Xph = np.zeros((len(diagnoses), len(patient_history_dict)))
    
    for i, phs in enumerate(patient_history):
        lst = str(phs).split(';')

        for ph in lst:
            Xph[i][[patient_history_dict == ph]] = 1


    Xd = np.zeros((len(diagnoses), len(diagnoses_dict)))
    
    for i, ds in enumerate(diagnoses):
        lst = str(ds).split(';')

        for d in lst:
            Xd[i][diagnoses_dict == d] = 1

    X = np.concatenate((Xs, Xph), axis=1)
    X = np.concatenate((X, Xd), axis=1)

    return X

In [44]:
print("admissions with symptoms: ", len(symptoms_raw[symptoms_raw == symptoms_raw]) / len(symptoms_raw))

admissions with symptoms:  0.5344411247083862


In [45]:
print("admissions with patient_history: ", len(patient_history_raw[patient_history_raw == patient_history_raw]) / len(patient_history_raw))

admissions with patient_history:  0.5189702451602342


In [46]:
print("admissions with diagnoses: ", len(diagnoses_raw[diagnoses_raw == diagnoses_raw]) / len(diagnoses_raw))

admissions with diagnoses:  0.99907911431261


In [47]:
symptoms_dict, patient_history_dict, diagnoses_dict  = load_diagnose_data()

print(f"symptoms: {len(symptoms_dict)}, patient_history: {len(patient_history_dict)}, diagnoses: {len(diagnoses_dict)}")

t_start = time.time()
diagnoses_matrix = diagnoses_to_matrix(symptoms_raw, patient_history_raw, diagnoses_raw, symptoms_dict[:,0], patient_history_dict[:,0], diagnoses_dict[:,0])
t_end = time.time()

shape = diagnoses_matrix.shape
ones = sum(sum(diagnoses_matrix))

print(f"Time elapsed: {(t_end - t_start) / 60:.2f} minutes")
print(f"diagnoses_matrix shape: {shape}")
print(f"percentage of ones: {100 * ones / (shape[0] * shape[1])}")

symptoms: 1141, patient_history: 470, diagnoses: 4730


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


Time elapsed: 0.72 minutes
diagnoses_matrix shape: (48866, 6341)
percentage of ones: 0.1776344906678388


In [48]:
if False:
  with open("/content/drive/MyDrive/uni/RoboDoc/ml/diagnoses_matrix_raw.npy", 'wb') as f:
    np.save(f, diagnoses_matrix)

  with open("/content/drive/MyDrive/uni/RoboDoc/ml/symptoms_dict.npy", 'wb') as f:
    np.save(f, symptoms_dict)

  with open("/content/drive/MyDrive/uni/RoboDoc/ml/patient_history_dict.npy", 'wb') as f:
    np.save(f, patient_history_dict)

  with open("/content/drive/MyDrive/uni/RoboDoc/ml/diagnoses_dict.npy", 'wb') as f:
    np.save(f, diagnoses_dict)

### Reduce dimensionality of diagnoses with PCA:

In [ ]:
from sklearn.decomposition import PCA
import pickle as pk

In [ ]:

pca = PCA(n_components=(1/100) * pca_prec)

t_start = time.time()
M = pca.fit_transform(diagnoses_matrix)
t_end = time.time()

with open(f"{df_path}/diagnoses_matrix_{df_name}.npy", 'wb') as f:
    np.save(f, M)

pk.dump(pca, open(f"{df_path}/pca_{df_name}.pkl", 'wb')) 
# pca_reload = pk.load(open("pca.pkl",'rb')) 

print(f"Time elapsed: {(t_end - t_start) / 60:.2f} minutes")
print(M.shape)

Time elapsed: 11.10 minutes
(48866, 1436)


In [ ]:
print(pca.explained_variance_ratio_)

[3.25687098e-02 3.07907631e-02 2.08289666e-02 ... 5.56442546e-05
 5.56242847e-05 5.55425765e-05]


In [ ]:
with open(f"{df_path}/diagnoses_matrix_{df_name}.npy", 'rb') as f:
    M = np.load(f)
  
print(M.shape)

(48866, 1436)


### Join DataFrame and diagnose matrix:

In [ ]:
df_diagnoses = pd.DataFrame(M)
df = df.reset_index(drop=True)
full_df = df.join(df_diagnoses)

print(df.shape)
print(df.columns)
print(full_df.shape)

(48866, 16)
Index(['age', 'gender', 'weight', 'meanbp_mean', 'meanbp_min', 'meanbp_max',
       'resprate_min', 'resprate_max', 'resprate_mean', 'tempc_mean',
       'glucose_min', 'glucose_max', 'glucose_mean', 'length_of_stay_hospital',
       'days_to_death', 'died_in_hospital'],
      dtype='object')
(48866, 1452)


## Normalize data:

In [ ]:
from sklearn import preprocessing

length_of_stay = full_df.pop("length_of_stay_hospital")
died_in_hospital = full_df.pop("died_in_hospital")
days_to_death = full_df.pop("days_to_death")

X = full_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)
normed_df = pd.DataFrame(X_scaled, columns=full_df.columns)

normed_df["length_of_stay_hospital"] = length_of_stay
normed_df["died_in_hospital"] = died_in_hospital
normed_df["days_to_death"] = days_to_death

normed_df.describe()


,age,gender,weight,meanbp_mean,meanbp_min,meanbp_max,resprate_min,resprate_max,resprate_mean,tempc_mean,glucose_min,glucose_max,glucose_mean,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,...,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435,length_of_stay_hospital,days_to_death
count,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,...,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,48866.000000,21444.000000
mean,0.650590,0.562129,0.266388,0.445591,0.467871,0.296353,0.283202,0.313857,0.301810,0.608499,0.180799,0.162328,0.120644,0.262751,0.404717,0.369815,0.386130,0.490819,0.443567,0.466785,0.455792,0.338298,0.435870,0.436951,0.453190,0.424337,0.405111,0.463086,0.427688,0.462124,0.394588,0.462222,0.465889,0.468876,0.499696,0.397292,0.443534,0.482537,0.492243,0.495614,...,0.493126,0.455065,0.485833,0.393557,0.442577,0.434469,0.455794,0.446471,0.488138,0.476483,0.459526,0.418764,0.458832,0.430521,0.447943,0.475411,0.464631,0.472509,0.451623,0.493592,0.408311,0.494183,0.380829,0.483436,0.434358,0.469886,0.486177,0.448833,0.485848,0.402913,0.468509,0.443845,0.470479,0.494462,0.457499,0.425124,0.426741,0.480849,9.990334,12.628474
std,0.228188,0.496130,0.072670,0.082701,0.109438,0.094120,0.086976,0.106607,0.102414,0.060794,0.058942,0.087584,0.045213,0.177362,0.173491,0.128753,0.133040,0.122872,0.116459,0.130797,0.117562,0.119592,0.122049,0.100812,0.108233,0.109583,0.093970,0.099768,0.124292,0.123155,0.125566,0.104172,0.093376,0.095048,0.094871,0.091836,0.093674,0.087280,0.100602,0.078760,...,0.042387,0.041216,0.040174,0.034711,0.044265,0.042217,0.034033,0.037864,0.036830,0.044027,0.046013,0.030993,0.039151,0.036317,0.037540,0.047838,0.031877,0.038163,0.039022,0.037780,0.029666,0.045057,0.033834,0.039320,0.039902,0.044196,0.033762,0.043657,0.045517,0.034513,0.033318,0.036002,0.039627,0.045321,0.041871,0.035925,0.037507,0.045871,10.812359,9.036426
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.000000
25%,0.500000,0.000000,0.220736,0.386861,0.408000,0.242754,0.228972,0.245902,0.230769,0.571816,0.145762,0.107252,0.091931,0.113990,0.265941,0.280532,0.324501,0.430862,0.377935,0.387450,0.398253,0.268717,0.371063,0.388484,0.415418,0.364886,0.354959,0.407442,0.366554,0.395420,0.331553,0.422746,0.408808,0.412807,0.463294

In [ ]:
pk.dump(min_max_scaler, open(f"{df_path}/scaler_{df_name}.pkl", 'wb')) 

## Save DataFrame:

In [ ]:
normed_df.to_pickle(f"{df_path}/dataframe_{df_name}.pkl")